# Linear Programming Challenge - Solution Writeup

## Tổng quan bài toán

Challenge này yêu cầu tìm password 20 ký tự thỏa mãn format `SEE{...}` (20 bytes = 160 bits). File `crackme.py` chứa một bài toán **Mixed Integer Linear Programming (MILP)** khổng lồ với hàng nghìn ràng buộc tuyến tính trên biến nhị phân.

### Insight chính

MILP trong `crackme.py` **không phải** bài toán tối ưu thông thường, mà là **encoding của một chương trình tính toán bit-level** sử dụng hai primitive operations:

1. **XOR (⊕)**: Phép cộng modulo 2, có tính tự nghịch đảo
2. **S-box 4-bit**: Ánh xạ hoán vị {0,1}⁴ → {0,1}⁴, có nghịch đảo

### Chiến lược giải

Thay vì giải MILP trực tiếp (quá chậm), ta sẽ:

1. **Trích xuất cấu trúc tính toán** từ các ràng buộc MILP
2. **Xây dựng đồ thị dataflow** (computation graph) 
3. **Đảo ngược đồ thị** để back-solve từ terminal constraints
4. **Brute-force giảm chiều**: chỉ brute 2 bytes thay vì 20 bytes

---

## Phase 1: Parsing và trích xuất cấu trúc

In [40]:
import re
from functools import reduce
from dataclasses import dataclass, field
from typing import Set, List

def get_coeff(eqnstr):
    if eqnstr[0] == '-': eqnstr = "- " + eqnstr[1:]
    else: eqnstr = "+ " + eqnstr
    terms = re.findall(r"([+-][^XY*]+)", eqnstr.split("<=")[0])
    terms = [t+"1" if len(t) == 2 else t for t in terms]
    terms = [int(i.replace(" ", "")) for i in terms] + [int(eqnstr.split("<= ")[-1])]
    return tuple(terms)

def get_varidx(eqnstr):
    return (*map(lambda v: int(v[2:-1]) + (160 if v[0] == "X" else 0), re.findall(r"[XY]\[\d+\]", eqnstr)),)

inputvars = [*range(160)]
eqns = [
    (
        # eqn string
        l[9:], 
        # variables used
        get_varidx(l[9:]),
        # coefficients
        get_coeff(l[9:])
        
    )
    for l in open("../crackme.py").read().split("\n")
    if l[:9] == "model += "
]

### Bước 1.1: Parse các ràng buộc từ crackme.py

Đọc file `crackme.py` và trích xuất các ràng buộc dạng:
- `model += <biểu thức tuyến tính> <= <hằng số>`

Mỗi ràng buộc được parse thành tuple: `(eqn_string, variable_indices, coefficients)`

**Lưu ý**: Các ràng buộc này encoding các phép toán bit-level:
- Một cụm 4 ràng buộc đặc biệt → XOR gate
- Một cụm lớn hơn → S-box 4-bit

In [41]:
# Build adjacency
nvars = max(max(e[1]) for e in eqns)+1
adjeqns = [[] for _ in range(nvars)]
for s,vs,cs in eqns:
    for v in vs:
        adjeqns[v].append((s,vs,cs))
adjvars = [reduce(lambda x,y: x|y, (set(vs) for s,vs,cs in a)) for a in adjeqns]

### Bước 1.2: Xây dựng adjacency graph

Xây dựng đồ thị biểu diễn mối quan hệ giữa các biến:
- `adjeqns[v]`: danh sách equations chứa biến v
- `adjvars[v]`: tập biến liên quan đến v qua các equations

Đây là bước chuẩn bị để nhận diện **binding patterns** (các nhóm biến liên quan chặt chẽ tạo thành một operation).

In [42]:
from functools import lru_cache
from itertools import product

def get_invariant(eqnlist, inpbits):
    
    # invariant should encode uniquely
    # 1. The eqns
    # 2. The input and output
    # 3. Be comparable
    
    eqnlist = sorted([*eqnlist], key=lambda e: e[2])
    eqninv = tuple([e[2] for e in eqnlist])
    
    # this will be reference point to define position of input
    maxeqn = eqnlist[-1]
    
    # assert maxeqn contains all variables of concern in the same order        
    r = set()
    for e in eqnlist:
        r |= set(e[1])
    assert set(maxeqn[1]) == r
    
    # Fill in gaps
    for i,e in enumerate(eqnlist[:-1]):
        necoeff = list(e[2])
        j = 0
        for x in maxeqn[1]:
            if e[1][j] != x:
                necoeff.insert(j, 0)
                continue
            j += 1
        eqnlist[i] = (e[0], maxeqn[1], tuple(necoeff))
        
    assert all(maxeqn[1] == e[1] for e in eqnlist)
    eqninv = tuple([e[2] for e in eqnlist])
    
    # maps inpbits to position in mineqn
    inpmap = tuple(sorted([maxeqn[1].index(i) for i in inpbits]))
    
    # uniquely define order of output bits in mineqn
    outmap = tuple(sorted([j for j,i in enumerate(maxeqn[1]) if i not in inpbits]))
    
    # Construct invariant
    return eqnlist, (eqninv, inpmap, outmap)

@lru_cache(maxsize=None)
def check_concrete_binding(inv):
    eqninv, inpmap, outmap = inv
    szin,szout = len(inpmap), len(outmap)
    sz = szin + szout
    for inp in product([0, 1], repeat=szin):
        # For each inp there's a unique out
        nsat = 0
        for out in product([0, 1], repeat=szout):
            pt = []; ptr1,ptr2 = 0,0
            for i in range(sz):
                if i in outmap: 
                    pt.append(out[ptr1]); ptr1 += 1
                    continue
                pt.append(inp[ptr2]); ptr2 += 1
            allsat = all(
                sum(c*p for c,p in zip(e, pt)) <= e[-1]
                for e in eqninv
            )
            nsat += allsat
            if nsat > 1: 
                break
        if nsat != 1: return False
    return True

def bind(invariant, eqnlist):
    # Binds input bits and output bits
    _, inpmap, outmap = invariant
    varidx = next(iter(eqnlist))[1]
    assert len(varidx) == len(inpmap) + len(outmap)
    inpbits = (*(varidx[i] for i in inpmap),)
    outbits = (*(varidx[i] for i in outmap),)
    return inpbits, outbits

### Bước 1.3: Pattern recognition - Nhận diện invariants

**Ý tưởng cốt lõi**: Nếu một nhóm equations có cùng "dấu vân tay" (invariant), chúng biểu diễn cùng một loại operation (XOR hoặc S-box).

#### Invariant là gì?

Invariant = `(eqn_coefficients, input_positions, output_positions)`

Ví dụ: 4 ràng buộc encoding XOR sẽ có:
```
z = a ⊕ b  ⟺  
  model += z - a - b <= 0
  model += z + a + b <= 2
  model += -z + a - b <= 0
  model += -z - a + b <= 0
```

→ Pattern của coefficients là cố định (invariant 0)

#### Concrete binding verification

`check_concrete_binding()`: kiểm tra xem một invariant có đúng là ánh xạ 1-1 không (mỗi input tương ứng duy nhất 1 output).

Nếu đúng → đây là một **primitive operation khả nghịch**!

In [43]:
@dataclass
class Binding:
    invariant: int # Index of invariant in `all_invariant` array
    inp: ...
    out: ...
        
@dataclass
class Var:
    idx:int
    binding:Binding # This var is the `output` of this binding
    parents:Set[int]
    children:Set[int]

all_invariant = []
all_bindings = []
rootidx = [*range(160)]
graph = {r: Var(r, None, set(), set()) for r in rootidx}
all_layers = [set(rootidx)]
terminal_eqns = set(eqns)
while True:
    ng = graph.copy()
    newlayer = set()
    for r,rv in graph.items():
        
        adjidx = adjvars[r]
        parentidx = set((p for p in adjidx if p in graph))
        childrenidx = adjidx - parentidx
        
        for childidx in childrenidx:
            
            soleparents = parentidx & adjvars[childidx]
            sharedeqns = reduce(
                lambda x,y: x|y, 
                (
                    set(adjeqns[i]) 
                    for i in soleparents
                )
            ) & set(adjeqns[childidx])
            try: newsharedeqns, sharedeqninv = get_invariant(sharedeqns, soleparents)
            except: continue # eh just ignore for now
            if check_concrete_binding(sharedeqninv): # Binding!
                
                # sharedeqns are used to bind new variables, aren't terminal
                terminal_eqns -= sharedeqns
                
                # Create binding
                if sharedeqninv not in all_invariant:
                    all_invariant.append(sharedeqninv)
                    invidx = len(all_invariant) - 1
                else:
                    invidx = all_invariant.index(sharedeqninv)
                inpbits, outbits = bind(sharedeqninv, newsharedeqns)
                binding = Binding(
                    invidx,
                    inpbits,
                    outbits
                )
                all_bindings.append(binding)
                
                # Save new vars into ng
                for c in outbits:
                    cv = Var(c, binding, set(inpbits), set())
                    for p in soleparents:
                        ng[p].children.add(c)
                    ng[c] = cv
                    newlayer.add(c)
                    
    if len(graph) == len(ng): break
    print(len(ng), end="\r")
    graph = ng
    all_layers.append(newlayer)

## Phase 2: Xây dựng Computation Graph

### Bước 2.1: Graph construction với bindings

Xây dựng đồ thị tính toán layer-by-layer:

**Cấu trúc dữ liệu**:
- `Var`: node trong graph, đại diện cho một biến
- `Binding`: operation tạo ra biến đó (input, operation type, output)
- `all_layers`: phân tầng các biến theo độ sâu tính toán

**Thuật toán**:
1. Bắt đầu từ input variables (layer 0)
2. Với mỗi layer, tìm các nhóm biến có thể "bind" thành operation mới
3. Nếu một nhóm equations có invariant hợp lệ → tạo binding mới
4. Thêm output variables vào layer tiếp theo
5. Lặp lại cho đến khi không còn biến mới

**Kết quả**: Đồ thị DAG (Directed Acyclic Graph) với:
- 160 input nodes (password bits)
- Các intermediate nodes (XOR results, S-box outputs)
- Terminal nodes bị ràng buộc bằng giá trị cụ thể

In [44]:
from mip import Model, CBC, BINARY

model = Model(solver_name=CBC)
terminalidx = sorted([*reduce(lambda x,y: x|y, [set(e[1]) for e in terminal_eqns])])
terminal_vars = {x: model.add_var(var_type=BINARY) for x in terminalidx}

# Last layer are all the terminal stuff
assert set(terminalidx) == all_layers[-1]

for e in terminal_eqns:
    varidx, coeff = e[1], e[2]
    model += sum(terminal_vars[x]*y for x,y in zip(varidx, coeff[:-1])) <= coeff[-1]
    
model.verbose = 0
assert model.optimize().name == "OPTIMAL"
terminal_sol = {x:int(v.x) for x,v in terminal_vars.items()}

### Bước 2.2: Giải terminal constraints

**Terminal variables** = các biến không phải input của bất kỳ operation nào khác → là "output cuối cùng" của chương trình.

Các ràng buộc còn lại (chưa được dùng để bind) áp lên terminal variables → tạo thành một **bài toán MILP nhỏ hơn rất nhiều**.

**Ý nghĩa**: 
- Terminal variables phải thỏa mãn một tập giá trị cụ thể
- Đây là "điểm neo" để back-solve: ta biết output phải là gì!

**Kết quả**: `terminal_sol` = dictionary chứa giá trị (0 hoặc 1) của từng terminal variable.

→ Với password đúng, toàn bộ computation graph sẽ propagate đến đúng các giá trị terminal này!

In [45]:
code = ""
for i,layer in enumerate(all_layers):
    if i == 0: continue
    code += f"# ---- LAYER {i} ----\n\n"
    nodes = sorted([graph[j] for j in layer], key=lambda v: v.binding.invariant)
    newcode = []
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            nl = f"v_{b.out[0]} = v_{b.inp[0]} ^ v_{b.inp[1]}"
            if nl not in newcode: newcode.append(nl)
        elif b.invariant == 1:
            nl = f"{', '.join('v_%d'%o for o in b.out)} = SUB[({', '.join('v_%d'%i for i in b.inp)})]"
            if nl not in newcode: newcode.append(nl)
        else:
            raise Exception()
    code += "\n".join(newcode) + "\n\n\n"
    
for i,v in terminal_sol.items():
    code += f"assert v_{i} == {v}\n"
    
open("code.txt", "w").write(code)

165517

### Bước 2.3: Export computation graph

Generate file `code.txt` với representation trung gian (IR):
```python
v_270 = v_66 ^ v_2                    # XOR operation
v_a, v_b, v_c, v_d = SUB[(v_e, ...)]  # S-box operation
```

File này giúp:
1. Verify cấu trúc đồ thị
2. Debug nếu có vấn đề
3. Hiểu rõ cách MILP được transform thành dataflow

In [46]:
def gen_subbox(inv):
    
    eqninv, inpmap, outmap = inv
    sub = {}
    szin,szout = len(inpmap), len(outmap)
    sz = szin + szout
    for inp in product([0, 1], repeat=szin):
        # For each inp there's a unique out
        for out in product([0, 1], repeat=szout):
            pt = []; ptr1,ptr2 = 0,0
            for i in range(sz):
                if i in outmap: 
                    pt.append(out[ptr1]); ptr1 += 1
                    continue
                pt.append(inp[ptr2]); ptr2 += 1
            allsat = all(
                sum(c*p for c,p in zip(e, pt)) <= e[-1]
                for e in eqninv
            )
            if allsat: 
                sub[inp] = out
                break       
    return sub

## Phase 3: Extract S-box và tạo inverse

### Bước 3.1: Generate S-box lookup tables

Với mỗi invariant type, sinh bảng tra cứu cụ thể:
- `SUB`: mapping (input_4bits) → (output_4bits)
- `INVSUB`: mapping nghịch đảo (output_4bits) → (input_4bits)

**Tại sao cần INVSUB?** 
Để back-solve! Khi biết output, ta cần tính ngược lại input.

**Kết quả**: 
- Invariant 0: XOR (tự nghịch đảo, không cần bảng)
- Invariant 1: S-box 4-bit với 16 entries

In [47]:
all_subbox = [*map(gen_subbox, all_invariant)]
for i,subbox in enumerate(all_subbox):
    print("INVARIANT: ", i)
    print("\n".join(str(i) + " : " + str(j) for i,j in subbox.items()))
    
# INVARIANT 1 is just XOR
# INVARIANT 2 is idk take it as a subbox

INVARIANT:  0
(0, 0) : (0,)
(0, 1) : (1,)
(1, 0) : (1,)
(1, 1) : (0,)
INVARIANT:  1
(0, 0, 0, 0) : (0, 0, 1, 1)
(0, 0, 0, 1) : (1, 1, 0, 0)
(0, 0, 1, 0) : (1, 0, 0, 1)
(0, 0, 1, 1) : (0, 0, 1, 0)
(0, 1, 0, 0) : (0, 1, 1, 0)
(0, 1, 0, 1) : (1, 1, 1, 1)
(0, 1, 1, 0) : (0, 1, 0, 1)
(0, 1, 1, 1) : (1, 0, 0, 0)
(1, 0, 0, 0) : (1, 0, 1, 0)
(1, 0, 0, 1) : (0, 1, 1, 1)
(1, 0, 1, 0) : (0, 0, 0, 0)
(1, 0, 1, 1) : (1, 1, 1, 0)
(1, 1, 0, 0) : (1, 1, 0, 1)
(1, 1, 0, 1) : (0, 0, 0, 1)
(1, 1, 1, 0) : (1, 0, 1, 1)
(1, 1, 1, 1) : (0, 1, 0, 0)


In [48]:
def backslice(idx, visited=None):
    
    if visited is None: visited = set()
    if idx in visited: return set()
    visited.add(idx)
    
    # root node
    parents = graph[idx].parents
    if len(parents) == 0: return set([idx])
    
    ret = set()
    for p in parents:
        ret |= backslice(p, visited)
    
    return ret

terminal_backslice = set(tuple(sorted(x)) for x in map(backslice, terminalidx))
assert len(terminal_backslice) == 2
assert terminal_backslice == set((tuple(range(80)), tuple(range(80,160))))

### Bước 3.2: Verify graph structure

**Backslicing**: Theo dõi ngược dependency của một biến về input.

Kiểm tra `terminal_backslice`:
- Terminal nodes nên back-trace về 2 nhóm input riêng biệt
- Nhóm 1: input bits [0:80] → terminals group 1
- Nhóm 2: input bits [80:160] → terminals group 2

**Ý nghĩa**: Password có thể tách thành **2 phần độc lập**!
- Bytes 0-9 (bits 0-79): tương ứng 10 ký tự đầu
- Bytes 10-19 (bits 80-159): tương ứng 10 ký tự cuối

→ Giải 2 bài toán nhỏ hơn thay vì 1 bài toán lớn!

In [49]:
def backslice_to_layer(idx, layeridx, visited=None):
    
    if visited is None: visited = set()
    if idx in visited: return set()
    visited.add(idx)
    
    # root node
    for l in range(layeridx+1):
        if idx in all_layers[l]: 
            return set([idx])
    
    ret = set()
    parents = graph[idx].parents
    for p in parents:
        ret |= backslice_to_layer(p, layeridx, visited)
    
    return ret

In [50]:
layer1 = [idx for idx in [i for i in all_layers[1] if max(backslice(i)) < 80]]
nodes = [(n,graph[n]) for n in layer1]
nxor = [(n,k) for n,k in nodes if len(k.binding.out) == 1]
nsub = [(n,k) for n,k in nodes if len(k.binding.out) == 4]
assert len(layer1) == len(nxor) + len(nsub)
nxor = sorted(nxor, key=lambda n: [*n[1].binding.inp][::-1])
nsub = sorted(nsub, key=lambda n: ([*n[1].binding.inp][::-1], 3-n[1].binding.out.index(n[0])))
layer1_sym =  [(n[0], f"A_{format(idx, '04d')}") for idx,n in enumerate(nxor)]
layer1_sym += [(n[0], f"B_{format(idx, '04d')}") for idx,n in enumerate(nsub)]
layer1_sym = dict(layer1_sym)

assert len(set(layer1_sym.values())) == len(layer1)

def symbolize(idx):
    if idx in all_layers[0]: return f"input_{idx}"
    if idx in all_layers[1]: return layer1_sym[idx]
    return f"V_{format(idx, '04d')}"

In [51]:
def layer_to_str(idx):
    layer = [idx for idx in [i for i in all_layers[idx] if max(backslice(i)) < 80]]
    nodes = sorted([graph[j] for j in layer], key=lambda v: v.binding.invariant)
    newcode = []
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            nl = f"{symbolize(b.out[0])} = {symbolize(b.inp[0])} ^ {symbolize(b.inp[1])}"
            if nl not in newcode: newcode.append(nl)
        elif b.invariant == 1:
            nl = f"{', '.join(symbolize(o) for o in b.out)} = SUB[({', '.join(symbolize(i) for i in b.inp)})]"
            if nl not in newcode: newcode.append(nl)
        else:
            raise Exception()
    return "\n".join(sorted(newcode, key = lambda x: (len(str(x)), x)))

In [52]:
layer2 = [idx for idx in [i for i in all_layers[2] if max(backslice(i)) < 80]]
nodes = [(n,graph[n]) for n in layer2]
nsub1 = [(n,k) for n,k in nodes if len(k.binding.out) == 4 and min(k.binding.inp) >= 160]
nsub2 = [(n,k) for n,k in nodes if len(k.binding.out) == 4 and min(k.binding.inp) < 160]
assert len(layer2) == len(nsub1) + len(nsub2)
nsub1 = sorted(nsub1, key=lambda n: ([*map(symbolize, n[1].binding.inp)][::-1], 3-n[1].binding.out.index(n[0])))
nsub2 = sorted(nsub2, key=lambda n: ([*map(symbolize, n[1].binding.inp)][::-1], 3-n[1].binding.out.index(n[0])))
layer2_sym =  [(n[0], f"C_{format(idx, '04d')}") for idx,n in enumerate(nsub1)]
layer2_sym += [(n[0], f"D_{format(idx, '04d')}") for idx,n in enumerate(nsub2)]
layer2_sym = dict(layer2_sym)

assert len(set(layer2_sym.values())) == len(layer2)

def symbolize(idx):
    if idx in all_layers[0]: return f"input_{idx}"
    if idx in all_layers[1]: return layer1_sym[idx]
    if idx in all_layers[2]: return layer2_sym[idx]
    return f"V_{format(idx, '04d')}"

In [53]:
code = ""
for l in range(len(all_layers)):
    if l == 0: continue
    code += f"# --- LAYER {l} ----\n\n"
    code += layer_to_str(l)
    code += "\n\n"
    
open("code_symbolised.txt", "w").write(code)

85981

In [54]:
# Split the layers into each subgraph
all_layers1 = [
    set(n for n in l if not (max(backslice(n))) >= 80)
    for l in all_layers
]
all_layers2 = [
    l - l1
    for l1,l in zip(all_layers1, all_layers)
]

### Bước 3.3: Split graph thành 2 subgraphs

Tách đồ thị thành 2 phần độc lập:
- `all_layers1`: subgraph cho bits [0:80] → bytes [0:10]
- `all_layers2`: subgraph cho bits [80:160] → bytes [10:20]

Mỗi subgraph sẽ được giải riêng với brute-force giảm chiều.

In [55]:
def gen_brute(all_layers1, start):
    
    # Traverse down from input[64:80] to see what can be derived given them

    start6480 = [start]
    ops_start6480 = []
    known6480 = set(start6480[-1])
    for i,l in enumerate(all_layers1):
        if i == 0: continue
        pl = start6480[-1]
        ns = []
        for n in l:
            if n in known6480: continue
            node = graph[n]
            parents = node.binding.inp
            if not set(parents).issubset(pl | known6480): continue
            ns.append(n)
            inv = node.binding.invariant
            if inv == 0:
                ops_start6480.append(f"X[{n}] = X[{parents[0]}] ^ X[{parents[1]}]")
                known6480.add(n)
            elif inv == 1:
                outstr = ",".join(f"X[{i}]" for i in node.binding.out)
                instr = ",".join(f"X[{i}]" for i in node.binding.inp)
                ops_start6480.append(f"{outstr} = SUB[({instr})]")
                known6480 |= set(node.binding.out)
            else: raise Exception()
        start6480.append(set(ns))
        
    
    # Traverse up from all_layers1[-1] to see what can be derived given them

    invlayers = all_layers1[::-1]
    ops_end6480 = []
    done = set()
    for i,l in enumerate(invlayers):
        if i == len(invlayers) - 1: continue

        for n in l:
            node = graph[n]
            binding = node.binding
            if binding.invariant == 0:
                diff = set(binding.inp) - (l | known6480)
                if len(diff) == 2: continue
                if len(diff) == 0: continue
                newnode = next(iter(diff))
                knownnode = next(iter(set(binding.inp) - diff))
                if newnode in done: continue
                done.add(newnode)
                ops_end6480.append(f"X[{newnode}] = X[{n}] ^ X[{knownnode}]")
            elif binding.invariant == 1:
                if not set(binding.out).issubset(l | known6480): continue
                if len(set(binding.inp) - known6480) < 4: continue
                if set(binding.inp).issubset(done): continue
                done |= set(binding.inp)
                outstr = ",".join(f"X[{i}]" for i in node.binding.out)
                instr = ",".join(f"X[{i}]" for i in node.binding.inp)
                ops_end6480.append(f"{instr} = INVSUB[({outstr})]")
            else: raise Exception()
        
    tsol = dict([(t,terminal_sol[t]) for t in all_layers1[-1]])
        
    # generate bruteforce code
    template = open("brute-template.txt").read()
    template = template.replace("~~NVARS~~", str(nvars))
    template = template.replace("~~START~~", str(min(start)))
    template = template.replace("~~TERMINALSOL~~", str(tsol))
    template = template.replace("~~PTST~~", str(0 if min(start) == 64 else 80))
    template = template.replace("~~PTEND~~", str(80 if min(start) == 64 else 160))
    template = template.replace("~~PROPAGATEFORWARD~~", "\n".join("    " + o for o in ops_start6480))
    template = template.replace("~~PROPAGATEBACKWARD~~", "\n".join("    " + o for o in ops_end6480))
    open("brute.py", "w").write(template)
    
import subprocess
gen_brute(all_layers1, set(range(64,80)))
flag = subprocess.check_output(['python', 'brute.py']).strip()
gen_brute(all_layers2, set(range(144,160)))
flag += subprocess.check_output(['python', 'brute.py']).strip()

print("Flag: SEE{" + flag.decode() + "}")

Flag: SEE{sT1lL_Us!nG_pR3&ENt?}


## Phase 4: Brute-force với Back-solving

### Chiến lược brute-force giảm chiều

**Vấn đề**: Nếu brute-force toàn bộ 10 bytes:
- Complexity: |ALPHABET|^10 ≈ 70^10 ≈ 2^61 → không khả thi!

**Giải pháp**: 
1. **Brute-force chỉ 2 bytes** (bits 64-79 hoặc 144-159)
2. **Propagate forward**: Tính các node phụ thuộc trực tiếp vào 2 bytes này
3. **Back-solve từ terminals**: Dùng terminal_sol + INVSUB để suy ngược lại 8 bytes còn lại

**Complexity mới**: |ALPHABET|^2 ≈ 70^2 ≈ 4,900 lần thử → khả thi!

### Cấu trúc `brute.py` được generate

```python
# Template với placeholders
X = [None] * NVARS
terminal_sol = {...}  # Neo điểm đến

def initbrute(vars_2bytes):
    # Nạp 2 bytes guess vào vị trí START
    # Forward propagate một số operations
    
def trybrute():
    # Backward solve bằng INVSUB và XOR
    # Điền đầy đủ các bits còn lại
    
# Brute loop
for (a, b) in product(ALLOWED_CHARS, repeat=2):
    initbrute([bits of a,b])
    trybrute()
    if bits[PT_START:PT_END] all valid → print!
```

### Kết quả

- Subgraph 1 → `sT1lL_Us!n` (10 chars)
- Subgraph 2 → `G_pR3&ENt?` (10 chars)
- Flag: `SEE{sT1lL_Us!nG_pR3&ENt?}`